# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [60]:
import pandas as pd
import numpy as np
from scipy import stats

from scipy.stats import chi2
from scipy.stats import norm

In [61]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [62]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [63]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [64]:
w = data[data.race=='w']
b = data[data.race=='b']


<div class="span5 alert alert-success">
<p> Your answers to Q1 and Q2 here </p>
1. We use a z-test for proportions here. The central limit theorem applies because we have a sufficient sample size and observations in each group, allowing us to assume that the mean is approximately normally distributed. 

2. The null hypothesis is that proportions for white and black calls are equal. The alternative hypothesis is that the proportions are not equal.
</div>
****

In [65]:
# Your solution to Q3 here

# Using bootstrap sampling
w_mean = w['call'].mean()
b_mean = b['call'].mean()
diff = w_mean - b_mean

w_bootstrap = [np.random.choice(w['call'] - w_mean, w.shape[0]) for _ in range(10000)]
b_bootstrap = [np.random.choice(b['call'] - b_mean, b.shape[0]) for _ in range(10000)]

w_bs_means = np.array(w_bootstrap).mean(axis=1)
b_bs_means = np.array(b_bootstrap).mean(axis=1)

bs_diffs = w_bs_means - b_bs_means
ci = np.percentile(bs_diffs, [2.5, 97.5])
p_value = (bs_diffs > w_mean - b_mean).mean()

print("-----Bootstrap sample-----")
print("Margin of error: {:.4f}".format(np.abs(ci[0])))
print("95% confidence interval: ({:.4f}, {:.4f})".format(*ci))
print("Observed difference: {:.4f}".format(diff))
print("p value: {} \n".format(p_value))
# No samples out of 10000 were found to have a greater mean difference
    
    
# Using z-test for proportions
w_var = w_mean * (1 - w_mean)
b_var = b_mean * (1 - b_mean)

w_samples = w.shape[0]
b_samples = b.shape[0]

ci = norm.ppf(np.array([0.025, 0.975])) * np.sqrt((w_var / w_samples) + (b_var / b_samples))

z_score = diff / np.sqrt((w_var / w_samples) + (b_var / b_samples))
p_value = 1 - norm.cdf(z_score)

print("-----z-test for proportions-----")
print("Margin of error: {:.4f}".format(np.abs(ci[0])))
print("95% confidence interval: ({:.4f}, {:.4f})".format(*ci))
print("Observed difference: {:.4f}".format(diff))
print("p_value: {}".format(p_value))

-----Bootstrap sample-----
Margin of error: 0.0152
95% confidence interval: (-0.0152, 0.0156)
Observed difference: 0.0320
p value: 0.0001 

-----z-test for proportions-----
Margin of error: 0.0153
95% confidence interval: (-0.0153, 0.0153)
Observed difference: 0.0320
p_value: 1.9312819064531972e-05


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
4. The percentage of people with white-sounding names were called back by recruiters significantly more than those with black-sounding names. This shows that race may play a big factor in an employer's decision to call candidates back.
5. Although we could see a significant difference in the race/name for callbacks, it's not necessarily the most important factor. There could be other factors that are more important, as well as factors that correlate with white-sounding names and black-sounding names which could also affect the decisions. To get a better understanding of what factors affect callbacks, we should analyze any correlations with other features as well.   
</div>
****
